Install packages

In [ ]:
install.packages(c('stringi', 'stringr', 'data.table', 'dplyr', 'gplots', 'openxlsx', 'tidyverse', 'BiocManager'))

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bitops’, ‘gtools’, ‘caTools’, ‘Rcpp’




In [ ]:
BiocManager::install(c("tximport", "GenomicRanges", "DESeq2"))

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.rstudio.com


Bioconductor version 3.15 (BiocManager 1.30.17), R 4.2.0 (2022-04-22)

Installing package(s) 'BiocVersion', 'tximport', 'GenomicRanges', 'DESeq2'

also installing the dependencies ‘formatR’, ‘plogr’, ‘png’, ‘Biostrings’, ‘matrixStats’, ‘lambda.r’, ‘futile.options’, ‘RSQLite’, ‘KEGGREST’, ‘XML’, ‘xtable’, ‘RCurl’, ‘GenomeInfoDbData’, ‘zlibbioc’, ‘MatrixGenerics’, ‘DelayedArray’, ‘futile.logger’, ‘snow’, ‘BH’, ‘AnnotationDbi’, ‘annotate’, ‘BiocGenerics’, ‘S4Vectors’, ‘IRanges’, ‘GenomeInfoDb’, ‘XVector’, ‘SummarizedExperiment’, ‘Biobase’, ‘BiocParallel’, ‘genefilter’, ‘locfit’, ‘geneplotter’, ‘RcppArmadillo’


Old packages: 'ggplot2', 'httr', 'roxygen2', 'tibble', 'xfun'



Load libraries

In [ ]:
library(stringi)
library(GenomicRanges)
library(DESeq2)
library(data.table)
library(dplyr)
library(stringr)
library(gplots)
library(tximport)
library(openxlsx)
library(tidyverse)

Check whether everything is loaded

In [ ]:
(.packages())

[1] "forcats"              "purrr"                "readr"               
 [4] "tidyr"                "tibble"               "ggplot2"             
 [7] "tidyverse"            "openxlsx"             "tximport"            
[10] "gplots"               "stringr"              "dplyr"               
[13] "data.table"           "DESeq2"               "SummarizedExperiment"
[16] "Biobase"              "MatrixGenerics"       "matrixStats"         
[19] "GenomicRanges"        "GenomeInfoDb"         "IRanges"             
[22] "S4Vectors"            "BiocGenerics"         "stats4"              
[25] "stringi"              "stats"                "graphics"            
[28] "grDevices"            "utils"                "datasets"            
[31] "methods"              "base"

Import Salmon output

In [ ]:
files <- paste0(c("89quant.genes.sf", "90quant.genes.sf", "91quant.genes.sf", "92quant.genes.sf", "94quant.genes.sf", "95quant.genes.sf", "96quant.genes.sf"))
names(files) <- paste0('sample', c(89, 90, 91, 92, 94, 95, 96))

txi.mut.wt <- tximport(files, "salmon", txOut = T, importer = read.delim)

cat("Number of transcripts in Data Frame :", nrow(txi.mut.wt$counts))
head(txi.mut.wt$counts)

1 
2 
3 
4 
5 
6 
7 




Number of transcripts in Data Frame : 13758

,sample89,sample90,sample91,sample92,sample94,sample95,sample96
FBgn0031292,232.000,25,8,12.000,164.000,2.239,11.000
FBgn0031289,30.000,4,10,22.000,5.000,5.000,6.000
FBgn0031288,22.000,6,10,13.000,10.000,4.000,6.000
FBgn0031282,83.870,13,40,60.000,41.358,35.000,74.000
FBgn0031279,49.458,7,0,4.452,43.733,0.000,1.316
FBgn0031275,66.000,10,26,19.001,21.000,20.000,30.000


Create colData matrix for DESeq2

In [ ]:
samples <- data.frame(row.names = names(files), sex = c("m", "m", "f", "f", "m", "f", "f"), condition = c("mut", "mut", "mut", "mut", "wt", "wt", "wt"))
# ddsTxi <- DESeqDataSetFromTximport(txi.mut.wt, colData = samples, design = ~condition + sex + condition:sex)
ddsTxi <- DESeqDataSetFromTximport(txi.mut.wt, colData = samples, design = ~sex + condition + sex:condition)
keep <- rowSums(counts(ddsTxi)) > 0

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
using counts and average transcript lengths from tximport



DESeq2 Multiple factor analysis

This is our experiment design

In [ ]:
ddsTxi$sex
ddsTxi$condition

colData(ddsTxi)

[1] m m f f m f f
Levels: f m

[1] mut mut mut mut wt  wt  wt 
Levels: wt mut

DataFrame with 7 rows and 2 columns
              sex condition
         <factor>  <factor>
sample89        m       mut
sample90        m       mut
sample91        f       mut
sample92        f       mut
sample94        m       wt 
sample95        f       wt 
sample96        f       wt 

Relevel data

In [ ]:
ddsTxi$condition <- relevel(ddsTxi$condition, "wt")
ddsTxi$sex <- relevel(ddsTxi$sex, "m")
ddsTxi$condition
ddsTxi$sex

[1] mut mut mut mut wt  wt  wt 
Levels: wt mut

[1] m m f f m f f
Levels: m f

In [ ]:
dds <- DESeq(ddsTxi)
resultsNames(dds)

estimating size factors

using 'avgTxLength' from assays(dds), correcting for library size

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Intercept"           "sex_f_vs_m"          "condition_mut_vs_wt"
[4] "sexf.conditionmut"

Understand how to use the 'contrast' option to output the right results

In [ ]:
qdds <- makeExampleDESeqDataSet(n=100,m=12)
qdds$genotype <- factor(rep(rep(c("I","II"),each=3),2))
qdds$genotype

design(qdds) <- ~ genotype + condition + genotype:condition
qdds <- DESeq(qdds) 
resultsNames(qdds)

# Note: design with interactions terms by default have betaPrior=FALSE

# the condition effect for genotype I (the main effect)
# results(qdds, contrast=c("condition","B","A"))

Divide results of DESeq2 to different variables

In [ ]:
res_ms <- results(dds, contrast=c("condition","mut","wt"))
res_fs <- results(dds, list(c("condition_mut_vs_wt","sexf.conditionmut")))
res_muts <- results(dds, list(c("sex_f_vs_m", "sexf.conditionmut")))
res_wts <- results(dds, contrast=c("sex", "f", "m"))

resultsNames(dds)

[1] "Intercept"           "sex_f_vs_m"          "condition_mut_vs_wt"
[4] "sexf.conditionmut"

Visualize data with plotMA

In [ ]:
pdf("ms_MAplot.pdf")
DESeq2::plotMA(res_ms, alpha = 0.05, ylim=c(-6, 6), cooks.Cutoff = FALSE)
dev.off()

pdf("fs_MAplot.pdf")
DESeq2::plotMA(res_fs, alpha = 0.05, ylim=c(-6, 6))
dev.off()

pdf("muts_MAplot.pdf")
DESeq2::plotMA(res_muts, alpha = 0.05, ylim=c(-6, 6))
dev.off()

pdf("wts_MAplot.pdf")
DESeq2::plotMA(res_wts, alpha = 0.05, ylim=c(-6, 6))
dev.off()

Warning message in plot.window(...):
“"cooks.Cutoff" is not a graphical parameter”
Warning message in plot.xy(xy, type, ...):
“"cooks.Cutoff" is not a graphical parameter”
Warning message in axis(side = side, at = at, labels = labels, ...):
“"cooks.Cutoff" is not a graphical parameter”
Warning message in axis(side = side, at = at, labels = labels, ...):
“"cooks.Cutoff" is not a graphical parameter”
Warning message in box(...):
“"cooks.Cutoff" is not a graphical parameter”
Warning message in title(...):
“"cooks.Cutoff" is not a graphical parameter”


png 
  2

png 
  2

png 
  2

png 
  2

In [ ]:
summary(res_ms, alpha = 0.05)
summary(res_fs, alpha = 0.05)
summary(res_muts, alpha = 0.05)
summary(res_wts, alpha = 0.05)


out of 13444 with nonzero total read count
adjusted p-value < 0.05
LFC > 0 (up)       : 158, 1.2%
LFC < 0 (down)     : 235, 1.7%
outliers [1]       : 0, 0%
low counts [2]     : 1823, 14%
(mean count < 6)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results


out of 13444 with nonzero total read count
adjusted p-value < 0.05
LFC > 0 (up)       : 300, 2.2%
LFC < 0 (down)     : 513, 3.8%
outliers [1]       : 0, 0%
low counts [2]     : 1823, 14%
(mean count < 6)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results


out of 13444 with nonzero total read count
adjusted p-value < 0.05
LFC > 0 (up)       : 270, 2%
LFC < 0 (down)     : 1534, 11%
outliers [1]       : 0, 0%
low counts [2]     : 0, 0%
(mean count < 0)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results


out of 13444 with nonzero total read count
adjusted p-value < 0.05
LFC > 0 (up)       : 188, 1.4%
LFC 

Shrink data (which helps to get rid of the noise according to M. Love)

In [ ]:
ms_resLFC <- lfcShrink(dds, coef = "conditions_mut_vs_wt", type = "apeglm",
                    lfcThreshold = 1)
fs_resLFC <- lfcShrink(dds, coef = "conditions_mut_vs_wt", type = "apeglm",
                    lfcThreshold = 1)
muts_resLFC <- lfcShrink(dds, coef = "conditions_mut_vs_wt", type = "apeglm",
                    lfcThreshold = 1)
wts_resLFC <- lfcShrink(dds, coef = "conditions_mut_vs_wt", type = "apeglm",
                    lfcThreshold = 1)

pdf("ms_MAplot_shrunk.pdf")
DESeq2::plotMA(ms_resLFC, alpha = 0.05, ylim=c(-6, 6))
dev.off()

pdf("fs_MAplot_shrunk.pdf")
DESeq2::plotMA(fs_resLFC, alpha = 0.05, ylim=c(-6, 6))
dev.off()

pdf("muts_MAplot_shrunk.pdf")
DESeq2::plotMA(muts_resLFC, alpha = 0.05, ylim=c(-6, 6))
dev.off()

pdf("wts_MAplot_shrunk.pdf")
DESeq2::plotMA(wts_resLFC, alpha = 0.05, ylim=c(-6, 6))
dev.off()
summary(res, alpha = 0.05)

Order gene expression data by lfc value and save as a table

In [ ]:
head(res_ms)

log2 fold change (MLE): condition mut vs wt 
Wald test p-value: condition mut vs wt 
DataFrame with 6 rows and 6 columns
             baseMean log2FoldChange     lfcSE       stat    pvalue      padj
            <numeric>      <numeric> <numeric>  <numeric> <numeric> <numeric>
FBgn0031292  63.00621     -1.2174969  0.597155 -2.0388292 0.0414671  0.461579
FBgn0031289   9.68462      1.0748568  1.289775  0.8333674 0.4046376  0.979068
FBgn0031288   9.76883      0.0914412  1.171870  0.0780301 0.9378041  0.998267
FBgn0031282  43.47912     -0.4380006  0.588830 -0.7438494 0.4569676  0.984515
FBgn0031279  21.87873     -1.3657309  0.950998 -1.4361035 0.1509729  0.793325
FBgn0031275  24.30377      0.2262799  0.819962  0.2759639 0.7825758  0.998267

In [ ]:
head(as.data.frame(res_ms))

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
FBgn0031292,63.006206,-1.21749694,0.5971550,-2.03882917,0.04146708,0.4615794
FBgn0031289,9.684616,1.07485678,1.2897754,0.83336740,0.40463756,0.9790677
FBgn0031288,9.768825,0.09144116,1.1718701,0.07803012,0.93780409,0.9982671
FBgn0031282,43.479117,-0.43800057,0.5888297,-0.74384937,0.45696761,0.9845153
FBgn0031279,21.878734,-1.36573092,0.9509976,-1.43610346,0.15097290,0.7933252
FBgn0031275,24.303772,0.22627992,0.8199619,0.27596394,0.78257576,0.9982671


In [ ]:
# transform IS NOT CURRENTLY POSSIBLE ON COLAB
# genes <- fread("dmel-all-r6.41_NOFASTA.gff", nThread = 10, showProgress = TRUE, verbose = TRUE) %>% 
  # dplyr::select(c(1, 4, 5, 7, 9)) %>% 
  # setNames(c("chr", "start", "end", "strand", "attr")) %>% 
  # mutate(id = sub('^ID=(FBgn[0-9]+);.*', '\\1', attr), chr = paste0("chr", chr), gene_name = sub('.*Name=([^;]+);.*', '\\1', attr), tss = ifelse(strand == "+", start, end)) %>% 
  #dplyr::select(-attr)

# ms_res.df <- as.data.frame(res_ms) %>% mutate(id = rownames(res_ms))
# fs_res.df <- as.data.frame(res_fs) %>% mutate(id = rownames(res_fs))
# muts_res.df <- as.data.frame(res_muts) %>% mutate(id = rownames(res_muts))
# wts_res.df <- as.data.frame(res_wts) %>% mutate(id = rownames(res_wts))

# ms_res.df <- merge(genes, ms_res.df, by = "id", all.y = T)
# fs_res.df <- merge(genes, fs_res.df, by = "id", all.y = T)
# muts_res.df <- merge(genes, muts_res.df, by = "id", all.y = T)
# wts_res.df <- merge(genes, wts_res.df, by = "id", all.y = T)
# write.xlsx(res.df, file = "base_results.xlsx", dec = ",")
# res.df %>% filter(is.na(gene_name)) %>% View

ms_res.df <- as.data.frame(res_ms)
fs_res.df <- as.data.frame(res_fs)
muts_res.df <- as.data.frame(res_muts)
wts_res.df <- as.data.frame(res_wts)

In [ ]:
library("dplyr")

In [ ]:
# filter
ms_ranking <- filter(ms_res.df, ms_res.df$padj < 0.05, abs(ms_res.df$log2FoldChange) > 0)
fs_ranking <- filter(fs_res.df, ms_res.df$padj < 0.05, abs(fs_res.df$log2FoldChange) > 0)
muts_ranking <- filter(muts_res.df, ms_res.df$padj < 0.05, abs(muts_res.df$log2FoldChange) > 0)
wts_ranking <- filter(wts_res.df, ms_res.df$padj < 0.05, abs(wts_res.df$log2FoldChange) > 0)

In [ ]:
# order final.data[order(final.data$age),]
ms_ordered_ranking <- ms_ranking[order(-ms_ranking$log2FoldChange),]
fs_ordered_ranking <- fs_ranking[order(-fs_ranking$log2FoldChange),]
muts_ordered_ranking <- muts_ranking[order(-muts_ranking$log2FoldChange),]
wts_ordered_ranking <- wts_ranking[order(-wts_ranking$log2FoldChange),]

setDT(ms_ordered_ranking, keep.rownames = "gene_id")
setDT(fs_ordered_ranking, keep.rownames = "gene_id")
setDT(muts_ordered_ranking, keep.rownames = "gene_id")
setDT(wts_ordered_ranking, keep.rownames = "gene_id")

In [ ]:
head(ms_ordered_ranking)

gene_id,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
FBgn0283462,130.34219,21.262287,5.948633,3.574315,3.511464e-04,1.416746e-02
FBgn0050031,260.24240,20.231270,5.948630,3.400997,6.714069e-04,2.492786e-02
FBgn0053872,825.16896,11.301623,2.062024,5.480839,4.233124e-08,5.071457e-06
FBgn0053852,254.29437,10.643743,2.315333,4.597069,4.284757e-06,3.222610e-04
FBgn0263121,152.07766,9.447224,2.081591,4.538464,5.666550e-06,3.919701e-04
FBgn0032286,77.51381,9.202157,2.094166,4.394186,1.111885e-05,6.909741e-04


In [ ]:
# save
write.xlsx(ms_ordered_ranking, file = "ms_ordered_ranking.xlsx", dec = ",")
write.xlsx(fs_ordered_ranking, file = "fs_ordered_ranking.xlsx", dec = ",")
write.xlsx(muts_ordered_ranking, file = "muts_ordered_ranking.xlsx", dec = ",")
write.xlsx(wts_ordered_ranking, file = "wts_ordered_ranking.xlsx", dec = ",")